<a href="https://colab.research.google.com/github/ChaejinE/MyPytorch/blob/main/f_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisite code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Hyperparameter
- Model Optimization 과정을 제어할 수 있는 조절가능 매개변수
- epoch : 데이터셋 반복 횟수
- batch size : 매개변수가 갱신 되기 전 신경망을 통해 전파된 데이터 샘플 수
- learning rate : 각 배치/에폭에서 모델의 학습속도를 조절

In [9]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

# loss function

In [4]:
loss_fn = nn.CrossEntropyLoss()

# Optimizer
- 최적화 절차(logic)은 optimizer 객체에 encapsulate된다.
- [다양한 Optimizer](https://pytorch.org/docs/stable/optim.html)
- Optimization 3단계
  - 1. optimizer.zero_grad()로 모델 매개 변수의 변화도를 재설정
    - 보통 gradient는 더해지므로 중복계산을 막기위해 반복할 때마다 명시적으로 0으로 설정
  - 2. loss.backward() 호출로 prediction loss를 역전파 시킨다. 
    - PyTorch는 각 매개변수에 대한 loss gradient를 저장한다.
  - 3. gradient 계산 뒤 optimizer.step() 호출로 역전파에서 수집된 변화도로 매개변수를 조정한다.

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # prediction & calculate loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error : \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("End")

Epoch 1
------------------
loss: 2.302563 [    0/60000]
loss: 2.294214 [ 6400/60000]
loss: 2.284707 [12800/60000]
loss: 2.282567 [19200/60000]
loss: 2.285628 [25600/60000]
loss: 2.266875 [32000/60000]
loss: 2.255532 [38400/60000]
loss: 2.242657 [44800/60000]
loss: 2.243642 [51200/60000]
loss: 2.240040 [57600/60000]
Test Error : 
 Accuracy: 46.6%, Avg loss: 2.229885 

Epoch 2
------------------
loss: 2.220282 [    0/60000]
loss: 2.213718 [ 6400/60000]
loss: 2.192281 [12800/60000]
loss: 2.220368 [19200/60000]
loss: 2.184751 [25600/60000]
loss: 2.167180 [32000/60000]
loss: 2.153036 [38400/60000]
loss: 2.122106 [44800/60000]
loss: 2.144888 [51200/60000]
loss: 2.137681 [57600/60000]
Test Error : 
 Accuracy: 52.2%, Avg loss: 2.118563 

Epoch 3
------------------
loss: 2.094565 [    0/60000]
loss: 2.080449 [ 6400/60000]
loss: 2.043118 [12800/60000]
loss: 2.116193 [19200/60000]
loss: 2.031405 [25600/60000]
loss: 2.010697 [32000/60000]
loss: 1.987612 [38400/60000]
loss: 1.932090 [44800/60000]
l